In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from pandas_datareader import data as wb
import matplotlib.pyplot as plt
from datetime import date
import math

In [2]:
tickers = pd.read_csv('TICKERS.csv')

tickers.shape

(5, 1)

In [3]:
def getFileNames(ticker):
    return 'C:\\Users\\leoml\\Documents\\Leonardo\\INVESTIMENTOS\\Ações Data\\balancos\\' + ticker.upper() + '.xls'

In [4]:
def is_nan(x):
    return (x is np.nan or x != x)
def adapt_date1(date):
    if (type(date) == float and is_nan(date)):
        return '0', 0, 0
    return date[6:] + date[3:5], int(date[6:]), int( int(date[3:5]) / 3 )
def adapt_date2(date):
    if (type(date) == float and is_nan(date)):
        return '0'
    return date[:4] + date[5:7] #, int(date[:4]), int( int(date[5:7]) / 3 )

In [5]:
def preprocessarBalanco(balancoFileName):
    
    balanco = pd.read_excel(balancoFileName,sheet_name="Bal. Patrim.",header=None,skiprows=1,dtype={0:'object'}).T
    balanco.iloc[0][0] = 'Data'
    
    cols = balanco.loc[0, :]
    balanco = balanco.drop(0, axis=0)
    balanco.columns = cols.values
    
    df = balanco
    df['Data'] = balanco['Data']
    
    datas = []
    anos = []
    trimestres = []
    for date in df['Data']:
        data, ano, trimestre = adapt_date1(date)
        datas.append(data)
        anos.append(ano)
        trimestres.append(trimestre)

    df['Data'] = datas
    df['Ano'] = anos
    df['Trimestre'] = trimestres
    
    df = df.astype('float64')
    df = df.astype({'Data':'int64','Ano':'int64','Trimestre':'int64'})
    
    return df.loc[df['Data'] != 0]

In [6]:
def preprocessarResultados(resultadoFileName):
    
    resultados = pd.read_excel(resultadoFileName,sheet_name="Dem. Result.",header=None,skiprows=1,dtype={0:'object'}).T
    resultados.iloc[0][0] = 'Data'
    
    cols = resultados.loc[0, :]
    resultados = resultados.drop(0, axis=0)
    resultados.columns = cols.values

    df = resultados
    df['Data'] = resultados['Data']

    datas = []
    for date in df['Data']:
        data, ano, trimestre = adapt_date1(date)
        datas.append(data)

    df['Data'] = datas
    
    df = df.astype('float64')
    df = df.astype({'Data':'int64'})
    
    return df

In [7]:
def preprocessarCotacoes(tickers,start_date, end_date):
    
    cotacoes = {}
    
    for t in tickers:
        print(t)
        cotacoes[t] = wb.DataReader((t + '.SA'), data_source='yahoo', start=start_date, end=end_date)
    
    return cotacoes

In [30]:
import urllib.request as rq
from bs4 import BeautifulSoup as soup
import pandas as pd

def get_fundamentus(ticker):

    def get_url(ticker):
        return 'https://www.fundamentus.com.br/detalhes.php?papel=' + ticker

    html = rq.urlopen(get_url(ticker))
    pretty_html = soup(html,'html.parser')
    tables = pretty_html.find_all('table',attrs={'class':'w728'})

    def interpret(tables):
        json = []
        for s1 in tables.text.split('\n\n\n'):
            for s2 in s1.split('\n?'):
                aux = s2.replace('?','').strip()
                if aux != '':
                    json.append(aux)
        return json

    info = []
    i = 0
    for t in tables:
        info.append(interpret(t))
        i+=1
    info

    json = {}

    for s in info[0]:
        aux = s.split('\n')
        json[aux[0]] = aux[1]

    for s in info[1]:
        aux = s.split('\n')
        json[aux[0]] = aux[1]

    for s in info[2][1:]:
        aux = s.split('\n')
        test = []
        for a in aux:
            b = a.strip()
            if b != '':
                test.append(b)
        json[test[0]] = test[1]

    for s in info[3][1:]:
        aux = s.split('\n')
        json[aux[0]] = aux[1]

    i = 0
    for s in info[4][2:]:
        aux = s.split('\n')
        if i % 2 == 0:
            json[(aux[0] + ' 12 meses')] = aux[1]
            i+=1
        else:
            json[(aux[0] + ' 3 meses')] = aux[1]
            i+=1

    for k in json.keys():
        if (json[k][0] in ['0','1','2','3','4','5','6','7','8','9']) and '/' not in json[k]:
            number = json[k].replace('.','')
            number = number.replace(',','.')
            if number[-1] != '%':
                number = float(number)
            else:
                number = float(number[:-1])
                number /= 100
            json[k] = number
    
    return json

get_fundamentus('SQIA3')

{'Papel': 'SQIA3',
 'Cotação': 17.54,
 'Tipo': 'ON',
 'Data últ cot': '14/11/2019',
 'Empresa': 'SINQIA ON',
 'Min 52 sem': 6.09,
 'Setor': 'Programas e Serviços',
 'Max 52 sem': 20.62,
 'Subsetor': 'Programas e Serviços',
 'Vol $ méd (2m)': 12095500.0,
 'Valor de mercado': 1237430000.0,
 'Últ balanço processado': '30/09/2019',
 'Valor da firma': 930940000.0,
 'Nro. Ações': 70549000.0,
 'Dia': 0.0233,
 'P/L': '-335,17',
 'LPA': '-0,05',
 'Mês': 0.0115,
 'P/VP': 2.87,
 'VPA': 6.11,
 '30 dias': 0.1693,
 'P/EBIT': 106.78,
 'Marg. Bruta': 0.33,
 '12 meses': 1.829,
 'PSR': 7.42,
 'Marg. EBIT': 0.069,
 '2019': 1.7753,
 'P/Ativos': 2.0,
 'Marg. Líquida': '-2,2%',
 '2018': '-4,53%',
 'P/Cap. Giro': 3.54,
 'EBIT / Ativo': 0.019,
 '2017': 0.9357,
 'P/Ativ Circ Liq': 5.89,
 'ROIC': 0.047,
 '2016': 0.5475,
 'Div. Yield': 0.001,
 'ROE': '-0,9%',
 '2015': 0.1106,
 'EV / EBITDA': 39.64,
 'Liquidez Corr': 8.38,
 '2014': '-21,96%',
 'Giro Ativos': 0.27,
 'Div Br/ Patrim': 0.14,
 'Cres. Rec (5a)': 0.241

In [8]:
balancos = {}
resultados = {}

for t in tickers['Ticker']:

    bal_res_name = getFileNames(t)
    print(bal_res_name)

    balancos[t] = preprocessarBalanco(bal_res_name)
    resultados[t] = preprocessarResultados(bal_res_name)

# cotacoes = preprocessarCotacoes(tickers['Ticker'], '2010-01-01', str(date.today()))

C:\Users\leoml\Documents\Leonardo\INVESTIMENTOS\Ações Data\balancos\IDNT3.xls
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
C:\Users\leoml\Documents\Leonardo\INVESTIMENTOS\Ações Data\balancos\ELET3.xls
WARNING *** file size (70002) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (70002) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
C:\Users\leoml\Documents\Leonardo\INVESTIMENTOS\Ações Data\balancos\PETR4.xls
WARNING *** file size (25125) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (25125) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
C:\Users\leoml\Documents\Leonardo\INVESTIMENTOS\

In [32]:
def medidas(ticker):
    df_b = balancos[ticker]
    df_r = resultados[ticker]
    
    df = df_b.set_index('Data').join(df_r.set_index('Data'))
    
    df['LUCRO_FUT'] = df['Lucro/Prejuízo do Período'].shift(-1) + df['Lucro/Prejuízo do Período'].shift(-2) + df['Lucro/Prejuízo do Período'].shift(-3) + df['Lucro/Prejuízo do Período'].shift(-4)

    df['SETOR'] = get_fundamentus(ticker)['Setor']
    
    df['TICKER'] = ticker
    
    return df.dropna()

# df = medidas('ABEV3')
# df

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 144 entries, 201909 to 201006
Data columns (total 84 columns):
Ativo Total                                                     144 non-null float64
Ativo Circulante                                                144 non-null float64
Caixa e Equivalentes de Caixa                                   144 non-null float64
Aplicações Financeiras                                          144 non-null float64
Contas a Receber                                                144 non-null float64
Estoques                                                        144 non-null float64
Ativos Biológicos                                               144 non-null float64
Tributos a Recuperar                                            144 non-null float64
Despesas Antecipadas                                            144 non-null float64
Outros Ativos Circulantes                                       144 non-null float64
Ativo Realizável a Longo Prazo       

In [34]:
import time
df = pd.DataFrame()
for t in tickers['Ticker']:
    df = pd.concat([df,medidas(t)])
    time.sleep(0.5)

In [36]:
df.shape

(144, 84)

In [40]:
pd.concat([df,pd.get_dummies(df['SETOR'])],axis=0)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


AttributeError: 'NoneType' object has no attribute 'is_extension'

In [37]:
def periodo_trimestre(ano, trimestre):
    start = ['-01-01','-04-01','-07-01','-10-01']
    end = ['-03-31','-06-30','-09-30','-12-31']
    return str(ano) + start[trimestre - 1], str(ano) + end[trimestre - 1]

In [ ]:
# data_corr = df.corr(method='spearman')
# sns.heatmap(data_corr, xticklabels=data_corr.columns, yticklabels=data_corr.columns, annot=True)

In [ ]:
# df.loc[np.isinf(df['RECEITAS']) | np.isinf(df['DESPESAS']) | np.isinf(df['LUCRO']) | np.isinf(df['ENDIVIDAMENTO']) | np.isinf(df['EBIT'])]

In [17]:
from sklearn.svm import SVR
from sklearn.linear_model import BayesianRidge, LinearRegression

reg = []

reg.append(('LinearRegression',LinearRegression(normalize=True)))


In [18]:
df = df.reset_index()
df = df.set_index(['TICKER','Data'])
X = df.drop(['LUCRO_FUT'],axis=1)
y = df['LUCRO_FUT']

In [25]:
df

Ativo Total  Ativo Circulante  Caixa e Equivalentes de Caixa  \
TICKER Data                                                                    
IDNT3  201909  6.639700e+04          4933.000                       3216.000   
       201906  6.403600e+04          2610.000                        817.000   
       201903  6.220500e+04          4197.000                       2388.000   
       201812  6.415100e+04          6028.000                       3594.000   
       201809  5.694000e+04          2122.000                        302.000   
       201806  6.110600e+04          2134.000                        309.000   
       201803  5.631200e+04          2974.000                        367.000   
       201712  5.524700e+04          2860.000                        323.000   
       201709  5.252100e+04          2786.000                        322.000   
       201706  6.027700e+04         14969.000                       1890.000   
       201703  7.676400e+04         22785.000                      12881.000   
       201612  9.503400e+04         24859.000                      13798.000   
       201609  2.200960e+05        120240.000                      36830.000   
       201606  2.320390e+05        130741.000                      36050.000   
       201603  2.468700e+05        148455.008                      26833.000   
       201512  3.420020e+05        142015.008                      31343.000   
       201509  4.741810e+05        341340.992                      25661.000   
       201506  6.185210e+05        415732.992                      21147.000   
       201503  6.419690e+05        424430.016                      17568.000   
       201412  7.314190e+05        484048.992                      50227.000   
       201409  7.516170e+05        466209.984                      32369.000   
       201406  6.974540e+05        423376.992                      12343.000   
       201403  7.337470e+05        486364.992                      14955.000   
       201312  8.928910e+05        640376.000                      75978.000   
       201309  7.867390e+05        538278.976                      27403.000   
       201306  7.524960e+05        496937.984                      23092.000   
       201303  7.665480e+05        565766.976                      98860.000   
       201212  8.666020e+05        690296.000                      66045.000   
       201209  8.195110e+05        648878.976                      36562.000   
       201206  7.461770e+05        563424.000                      42465.000   
...                     ...               ...                            ...   
ENBR3  201709  2.006511e+07       5172294.144                    1453691.008   
       201706  1.933049e+07       4610498.048                    2027593.984   
       201703  1.924367e+07       4620961.792                    1975059.968   
       201612  1.924383e+07       4663745.024                    2017709.056   
       201609  1.973519e+07       5285773.824                    2477019.904   
       201606  1.943531e+07       5395280.896                    2562443.008   
       201603  1.856334e+07       4830133.248                    1712295.936   
       201512  1.841234e+07       4863321.088                    1149997.952   
       201509  1.805723e+07       4839811.072                    1271280.000   
       201506  1.830229e+07       4847626.240                    1635022.976   
       201503  1.425342e+07       4329297.920                    1764196.992   
       201412  1.317180e+07       3123619.072                     826971.008   
       201409  1.309540e+07       3307674.880                    1397732.992   
       201406  1.304111e+07       3200129.024                    1562137.984   
       201403  1.461003e+07       5217838.080                     780131.008   
       201312  1.414149e+07       4706944.000                     924145.984   
       201309  1.366957e+07       2736867.072                     954838.016   
       201306  1

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import ShuffleSplit

cv = ShuffleSplit(n_splits=5, test_size=0.1, random_state=1)
for clf in reg:
    print(clf[0],(cross_val_score(clf[1], X, y, cv=cv, error_score='raise')))

LinearRegression [-2.01527361e+07 -1.70279300e+00  4.28464618e-01 -4.67846680e-01
 -1.31257767e+01]


In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.1)

In [ ]:
# print(X_train)
# print(X_test)
# print(y_train)
# print(y_test)

In [ ]:
from sklearn import preprocessing
# Get column names first
names = X.columns
# Create the Scaler object
scaler = preprocessing.StandardScaler()
# Fit your data on the scaler object
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train = pd.DataFrame(X_train, columns=names)

In [21]:
for clf in reg:
    clf[1].fit(X_train,y_train)

In [22]:
predict = []
for clf in reg:
    predict.append((clf[0],(clf[1].predict(X_test))))
predict.append(('Gabarito',y_test.to_list()))

In [ ]:
# for i in predict:
#     print(i)

In [23]:
ans = ''
for i in range(0,len(predict)):
    ans += ( '{:18s}'.format(predict[i][0]) )
    ans += '['
    first = True
    for a in range(0,len(predict[i][1])):
        if not first:
             ans += ( ', ' + '{:05.2f}'.format(predict[i][1][a]) )
        else:
             ans += ( '{:05.2f}'.format(predict[i][1][a]) )
             first = False
    ans += ']\n'
print(ans)

LinearRegression  [10298651.94, -7270613.53, 16905984.55, -11883610.49, -1189123.29, -37803.56, -2181783.18, -135234.80, 1489775.30, 1057946.50, -225004.19, 455136.15, -85546993035.72, -1874897.26, 15271722.08]
Gabarito          [12027970.82, 11945874.05, 11170565.63, 3425898.43, -5288516.10, 7309.00, -611686.05, -18262.00, 611854.96, 1272833.02, -209336.98, -65988.00, 3675182.91, 375768.01, 3108466.88]



In [ ]:
X_train